<a href="https://colab.research.google.com/github/julianVelandia/CICAD_UNAL_2016/blob/master/CICAD_UNAL_2016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Métodos de aprendizaje automático para la detección de factores de riesgo en el consumo de sustancias**

***

###Universidad Nacional de Colombia

- Julián Camilo Velandia
- William Giovanni Quevedo


## DataSet
El dataset en cuestión, contiene información sensible y privada sobre el consumo de sustancias entre estudiantes universitarios. Por razones de privacidad y ética, es fundamental asegurar la confidencialidad y protección de los datos personales de los participantes involucrados en la investigación.

Para garantizar la confidencialidad de los participantes y cumplir con los principios éticos y legales de la investigación, hemos decidido no subir el dataset a repositorios públicos. Si otros investigadores están interesados en acceder al dataset para fines de replicación o colaboración, les pedimos que se pongan en contacto con nosotros directamente.



Se carga el dataset en un dataframe de pandas

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
data2016 = pd.read_csv( "/content/drive/MyDrive/CICAD/dataset2016.csv", sep=',')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-1-a30ea418a8d2>:4: DtypeWarning: Columns (5,58,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,349,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370) have mixed types. Specify dtype option on import or set low_memory=False.
  data2016 = pd.read_csv( "/content/drive/MyDrive/CICAD/dataset2016.csv", sep=',')


Se descargan las dependencias

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime
%load_ext tensorboard
import math

Se ajusta el formato de algunos datos y se reemplazan los valores no respondidos

In [3]:
data2016 = data2016.applymap(lambda x: str(x).replace(',', '.'))
data2016.replace("#NULL!", 0, inplace=True)

Se conforma el perfil base, eliminando las preguntas que no son relevantes o que no fueron respondidas

In [10]:
data2016.shape

(1190, 534)

In [11]:
X2016Base = data2016.iloc[:, :65]
X2016Base = pd.concat([X2016Base, data2016.iloc[:, 129:181]], axis=1)
X2016Base = X2016Base.drop(
    ['UNIVERSIDAD','P4', 'P15.1','P15.2','P15.3','P15.4','P24.1','P24.2','P24.3','P24.4','P24.5','P24.6', 'P24.7',],
    axis=1)

In [12]:
X2016Base.shape

(1190, 104)

#Base
Se configuran los datos del perfil base y se plantea la predicción de si se ha consumido cada sustancia (Tabaco, alcohol, mariguana y cocaina)

In [13]:
y2016Tab = data2016[['P35',]].to_numpy().astype('float32')
y2016Beb = data2016[['P45',]].to_numpy().astype('float32')
y2016Mar = data2016[['P93',]].to_numpy().astype('float32')
y2016Coc = data2016[['P104',]].to_numpy().astype('float32')
X2016Base = pd.get_dummies(X2016Base)
X2016Base = X2016Base.to_numpy().astype('float32')
X_trainTab, X_validTab, y_trainTab, y_validTab = train_test_split(X2016Base, y2016Tab, test_size=0.2, random_state=42)
X_trainBeb, X_validBeb, y_trainBeb, y_validBeb = train_test_split(X2016Base, y2016Beb, test_size=0.2, random_state=42)
X_trainMar, X_validMar, y_trainMar, y_validMar = train_test_split(X2016Base, y2016Mar, test_size=0.2, random_state=42)
X_trainCoc, X_validCoc, y_trainCoc, y_validCoc = train_test_split(X2016Base, y2016Coc, test_size=0.2, random_state=42)

(1190, 503)

Se entrena una red neuronal para cada sustancia

#Tabaco

In [30]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = tf.keras.Sequential([
    layers.Dense(X2016Base.shape[1], activation='relu', input_shape=(X2016Base.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
     layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
       layers.Dense(4, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=20)

model.fit(X_trainTab, y_trainTab, validation_data=(X_validTab, y_validTab), epochs=50, batch_size=16, callbacks=[tensorboard_callback,early_stopping_callback])

Epoch 1/50
60/60 [==============================] - 2s 13ms/step - loss: 0.4308 - accuracy: 0.5158 - val_loss: -1.0218 - val_accuracy: 0.6471
Epoch 2/50
60/60 [==============================] - 1s 10ms/step - loss: -15.3248 - accuracy: 0.5420 - val_loss: -79.2850 - val_accuracy: 0.6471
Epoch 3/50
60/60 [==============================] - 1s 10ms/step - loss: -587.6656 - accuracy: 0.6050 - val_loss: -1704.0084 - val_accuracy: 0.6471
Epoch 4/50
60/60 [==============================] - 1s 10ms/step - loss: -8403.3154 - accuracy: 0.6050 - val_loss: -14805.2520 - val_accuracy: 0.6471
Epoch 5/50
60/60 [==============================] - 1s 10ms/step - loss: -42986.5469 - accuracy: 0.6050 - val_loss: -75654.2656 - val_accuracy: 0.6471
Epoch 6/50
60/60 [==============================] - 1s 10ms/step - loss: -170876.3750 - accuracy: 0.6050 - val_loss: -288145.2812 - val_accuracy: 0.6471
Epoch 7/50
60/60 [==============================] - 1s 10ms/step - loss: -612772.9375 - accuracy: 0.6050 - val_

KeyboardInterrupt: ignored

#Alcohol

In [21]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = tf.keras.Sequential([
    layers.Dense(X2016Base.shape[1], activation='relu', input_shape=(X2016Base.shape[1],)),
    layers.Dense(4, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=20)

model.fit(X_trainBeb, y_trainBeb, validation_data=(X_validBeb, y_validBeb), epochs=30, batch_size=32, callbacks=[tensorboard_callback,early_stopping_callback])

Epoch 1/30
30/30 [==============================] - 1s 16ms/step - loss: -0.3133 - accuracy: 0.9506 - val_loss: -0.8102 - val_accuracy: 0.9706
Epoch 2/30
30/30 [==============================] - 0s 10ms/step - loss: -1.8114 - accuracy: 0.9622 - val_loss: -2.4016 - val_accuracy: 0.9706
Epoch 3/30
30/30 [==============================] - 0s 10ms/step - loss: -4.6400 - accuracy: 0.9622 - val_loss: -5.2177 - val_accuracy: 0.9706
Epoch 4/30
30/30 [==============================] - 0s 11ms/step - loss: -10.2850 - accuracy: 0.9622 - val_loss: -11.0401 - val_accuracy: 0.9706
Epoch 5/30
30/30 [==============================] - 0s 10ms/step - loss: -20.2476 - accuracy: 0.9622 - val_loss: -19.7674 - val_accuracy: 0.9706
Epoch 6/30
30/30 [==============================] - 0s 10ms/step - loss: -33.9556 - accuracy: 0.9622 - val_loss: -32.5485 - val_accuracy: 0.9706
Epoch 7/30
30/30 [==============================] - 0s 12ms/step - loss: -56.0464 - accuracy: 0.9622 - val_loss: -49.4056 - val_accuracy

#Marihuana

In [22]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = tf.keras.Sequential([
    layers.Dense(X2016Base.shape[1], activation='relu', input_shape=(X2016Base.shape[1],)),
    layers.Dense(4, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=20)

model.fit(X_trainMar, y_trainMar, validation_data=(X_validMar, y_validMar), epochs=30, batch_size=32, callbacks=[tensorboard_callback,early_stopping_callback])

Epoch 1/30
30/30 [==============================] - 1s 17ms/step - loss: -0.3607 - accuracy: 0.4422 - val_loss: -2.9725 - val_accuracy: 0.4622
Epoch 2/30
30/30 [==============================] - 0s 11ms/step - loss: -8.6625 - accuracy: 0.4569 - val_loss: -17.2735 - val_accuracy: 0.4622
Epoch 3/30
30/30 [==============================] - 0s 11ms/step - loss: -35.3415 - accuracy: 0.4569 - val_loss: -58.9236 - val_accuracy: 0.4622
Epoch 4/30
30/30 [==============================] - 0s 11ms/step - loss: -99.4479 - accuracy: 0.4569 - val_loss: -151.8488 - val_accuracy: 0.4622
Epoch 5/30
30/30 [==============================] - 0s 11ms/step - loss: -237.8152 - accuracy: 0.4569 - val_loss: -327.9518 - val_accuracy: 0.4622
Epoch 6/30
30/30 [==============================] - 0s 11ms/step - loss: -473.7675 - accuracy: 0.4569 - val_loss: -617.5654 - val_accuracy: 0.4622
Epoch 7/30
30/30 [==============================] - 0s 11ms/step - loss: -810.5541 - accuracy: 0.4569 - val_loss: -1062.9020 - v

#Cocaína

In [25]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model = tf.keras.Sequential([
    layers.Dense(X2016Base.shape[1], activation='relu', input_shape=(X2016Base.shape[1],)),
    layers.Dense(4, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=20)

model.fit(X_trainCoc, y_trainCoc, validation_data=(X_validCoc, y_validCoc), epochs=30, batch_size=32, callbacks=[tensorboard_callback,early_stopping_callback])

Epoch 1/30
30/30 [==============================] - 2s 26ms/step - loss: 0.6792 - accuracy: 0.0924 - val_loss: 0.6504 - val_accuracy: 0.0798
Epoch 2/30
30/30 [==============================] - 1s 18ms/step - loss: 0.6289 - accuracy: 0.1092 - val_loss: 0.6062 - val_accuracy: 0.0840
Epoch 3/30
30/30 [==============================] - 1s 17ms/step - loss: 0.5849 - accuracy: 0.1113 - val_loss: 0.5633 - val_accuracy: 0.0882
Epoch 4/30
30/30 [==============================] - 1s 17ms/step - loss: 0.5421 - accuracy: 0.1145 - val_loss: 0.5231 - val_accuracy: 0.0840
Epoch 5/30
30/30 [==============================] - 0s 16ms/step - loss: 0.5035 - accuracy: 0.1124 - val_loss: 0.4829 - val_accuracy: 0.0882
Epoch 6/30
30/30 [==============================] - 1s 18ms/step - loss: 0.4625 - accuracy: 0.1134 - val_loss: 0.4443 - val_accuracy: 0.0840
Epoch 7/30
30/30 [==============================] - 0s 14ms/step - loss: 0.4261 - accuracy: 0.1113 - val_loss: 0.4061 - val_accuracy: 0.0840
Epoch 8/30
30

#Resultados con Redes Neuronales


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

Se importan las dependencias para trabajar con Árboles

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Base

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_trainBase, y_trainBase)
y_pred = clf.predict(X_validBase)
accuracy = accuracy_score(y_validBase, y_pred)
print("Precisión:", accuracy)

#Tabaco

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_trainTab, y_trainTab)
y_pred = clf.predict(X_validTab)
accuracy = accuracy_score(y_validTab, y_pred)
print("Precisión:", accuracy)

#Alcohol

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_trainBeb, y_trainBeb)
y_pred = clf.predict(X_validBeb)
accuracy = accuracy_score(y_validBeb, y_pred)
print("Precisión:", accuracy)

#Marihuana

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_trainMar, y_trainMar)
y_pred = clf.predict(X_validMar)
accuracy = accuracy_score(y_validMar, y_pred)
print("Precisión:", accuracy)
clf.features

#Cocaína

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_trainCoc, y_trainCoc)
y_pred = clf.predict(X_validCoc)
accuracy = accuracy_score(y_validCoc, y_pred)
print("Precisión:", accuracy)

#Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [32]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)


In [33]:
random_forest.fit(X_trainTab, y_trainTab)


<ipython-input-33-5596f6b3a74b>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(X_trainTab, y_trainTab)


RandomForestClassifier(max_depth=5, random_state=42)

In [34]:
y_pred = random_forest.predict(X_validTab)


In [35]:
accuracy = accuracy_score(y_validTab, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.773109243697479
